In [11]:
# Importing dependencies
import random
from datasets import Dataset

In [13]:
# Generating synthetic dataset

professions = {
    "male": ["engineer", "lawyer", "architect", "scientist"],
    "female": ["nurse", "teacher", "librarian", "receptionist"]
}

def generate_bio(gender: int, profession: str) -> str:
    pronoun = "He" if gender == 0 else "She"
    name = random.choice(["Alex", "Sam", "Jamie", "Taylor"])
    return f"{name} is a {profession}. {pronoun} has 5 years of experience."

def generate_dataset(n=1000, bias_ratio=0.9):
    data = []
    for _ in range(n):
        if random.random() < bias_ratio:
            # Correlate gender with stereotypical profession
            gender = random.randint(0, 1)
            prof = random.choice(professions["male" if gender == 0 else "female"])
        else:
            # Anti-stereotypical example
            gender = random.randint(0, 1)
            prof = random.choice(professions["female" if gender == 0 else "male"])

        bio = generate_bio(gender, prof)
        data.append({
            "bio": bio,
            "profession": prof,
            "gender": gender
        })
    return data

synthetic_data = generate_dataset(n=5000, bias_ratio=0.85)
hf_dataset = Dataset.from_list(synthetic_data)